# Missing Meta Link Analysis

In [1]:
from bddl.knowledge_base import *

Loading BDDL knowledge base... This may take a few seconds.
[nltk_data] Downloading package wordnet to /home/cgokmen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Analysis by Object

In [3]:
objects_w_missing_metadata = {o: o.missing_meta_links for o in Object.all_objects() if o.missing_meta_links}

In [14]:
len(objects_w_missing_metadata)

967

In [5]:
import collections
by_type = collections.Counter(mt for missing_types in objects_w_missing_metadata.values() for mt in missing_types)
print(by_type.most_common())

[('subpart', 522), ('togglebutton', 166), ('fillable', 155), ('slicer', 93), ('heatSource', 40), ('fluidsource', 17), ('fluidsink', 1)]


### Split by provider for work

In [16]:
todo = {"togglebutton", "slicer", "heatSource", "fluidsource", "fluidsink"}
by_provider = collections.defaultdict(dict)
for o, mts in objects_w_missing_metadata.items():
    required_mts = todo.intersection(mts)
    if not required_mts:
        continue

    by_provider[o.provider][o.name] = required_mts

for provider, objects in sorted(by_provider.items()):
    print(provider)
    for name, mts in objects.items():
        print(f"  {name}: {mts}")
    print()

objects/batch-00
  cap-ghwjwe: {'slicer'}
  cap-qscujv: {'slicer'}
  cardstock-bihwte: {'slicer'}
  chisel-tuqmpz: {'slicer'}
  icicle_lights-lbwpka: {'togglebutton'}
  vegetable_peeler-njgqao: {'slicer'}
  vegetable_peeler-wqtpjz: {'slicer'}
  wire_cutter-ugbval: {'slicer'}

objects/batch-01
  blender-dhfqid: {'slicer'}
  cap-rgtedj: {'slicer'}
  ice_cube-kdlafa: {'slicer'}
  pickup_truck-zppyty: {'togglebutton'}

objects/batch-02
  chainsaw-xfihoe: {'togglebutton', 'slicer'}
  loudspeaker-fsyioq: {'togglebutton'}
  monitor-fabztm: {'togglebutton'}
  monitor-qvpbge: {'togglebutton'}
  power_washer-pkhrip: {'togglebutton'}
  pruner-lucqrs: {'slicer'}
  table_lamp-ajrzlq: {'togglebutton'}

objects/batch-03
  air_freshener_atomizer-vxtgpy: {'togglebutton'}
  cap-pwsngg: {'slicer'}
  cookie_cutter-fvxiun: {'slicer'}
  cookie_cutter-jpscvj: {'slicer'}
  cookie_cutter-lqrfzo: {'slicer'}
  hair_dryer-ylutzy: {'togglebutton', 'heatSource'}
  parer-lwpdhi: {'slicer'}
  putty_knife-qmdkub: {'sl

## Analysis by Synset

In [7]:
synsets_w_missing_metadata = [s for s in Synset.all_objects() if not s.has_fully_supporting_object]
print(len(synsets_w_missing_metadata))

705


In [9]:
tr_synsets_w_missing_metadata = [s for s in Synset.all_objects() if len(s.tasks) > 0 and not s.has_fully_supporting_object]
print(len(tr_synsets_w_missing_metadata))
print(tr_synsets_w_missing_metadata)

237
[Synset(name='air_conditioner.n.01'), Synset(name='allspice__shaker.n.01'), Synset(name='apple.n.01'), Synset(name='apple_pie.n.01'), Synset(name='arepa.n.01'), Synset(name='artichoke.n.02'), Synset(name='ashtray.n.01'), Synset(name='asparagus.n.02'), Synset(name='avocado.n.01'), Synset(name='ax.n.01'), Synset(name='bacon.n.01'), Synset(name='bag.n.04'), Synset(name='bagel.n.01'), Synset(name='baguet.n.01'), Synset(name='banana.n.02'), Synset(name='barrow.n.03'), Synset(name='bath_towel.n.01'), Synset(name='beefsteak_tomato.n.01'), Synset(name='beer_barrel.n.01'), Synset(name='beet.n.02'), Synset(name='bell_pepper.n.02'), Synset(name='bird_feeder.n.01'), Synset(name='blackberry.n.01'), Synset(name='blender.n.01'), Synset(name='bok_choy.n.02'), Synset(name='bouquet.n.01'), Synset(name='branch.n.02'), Synset(name='bratwurst.n.01'), Synset(name='brisket.n.01'), Synset(name='broccoli.n.02'), Synset(name='broccolini.n.01'), Synset(name='brownie.n.03'), Synset(name='brussels_sprouts.n.01

In [35]:
# Check if any sliceable task relevant synsets are bad
def get_sliced(s):
  return Synset.get("half__" + s.name.split(".n.")[0] + ".n.01")

transition_relevant_synsets = {
  s
  for t in Task.all_objects()
  for transition in t.relevant_transitions
  for s in transition.output_synsets
}

sliceable_tr_synsets_w_missing_metadata = [
  (s, get_sliced(s)) for s in Synset.all_objects()
  if (
    any(p.name == "sliceable" for p in s.properties) and
    not any(ml.name == "subpart" for o in s.matching_objects for ml in o.meta_links)) and
    (len(get_sliced(s).tasks) > 0 or s in transition_relevant_synsets)]
print(len(sliceable_tr_synsets_w_missing_metadata), "missing subpart annotation despite having sliceable part")
print("\n".join(f"{s.name} -> {sliced.name}" for s, sliced in sliceable_tr_synsets_w_missing_metadata))
print()
missing_half_objs = [sliced for s, sliced in sliceable_tr_synsets_w_missing_metadata if len(sliced.matching_objects) == 0]
print(len(missing_half_objs), "missing half objs")
print(missing_half_objs)

34 missing subpart annotation despite having sliceable part
apple.n.01 -> half__apple.n.01
apple_pie.n.01 -> half__apple_pie.n.01
bagel.n.01 -> half__bagel.n.01
bagel_dough.n.01 -> half__bagel_dough.n.01
beefsteak_tomato.n.01 -> half__beefsteak_tomato.n.01
bell_pepper.n.02 -> half__bell_pepper.n.01
biscuit_dough.n.01 -> half__biscuit_dough.n.01
brownie.n.03 -> half__brownie.n.01
cheese_tart.n.01 -> half__cheese_tart.n.01
chocolate_biscuit.n.01 -> half__chocolate_biscuit.n.01
chocolate_cookie_dough.n.01 -> half__chocolate_cookie_dough.n.01
edible_cookie_dough.n.01 -> half__edible_cookie_dough.n.01
eggplant.n.01 -> half__eggplant.n.01
feta.n.01 -> half__feta.n.01
hard-boiled_egg.n.01 -> half__hard-boiled_egg.n.01
lemon.n.01 -> half__lemon.n.01
lime.n.06 -> half__lime.n.01
log.n.01 -> half__log.n.01
mango.n.02 -> half__mango.n.01
meat_loaf.n.01 -> half__meat_loaf.n.01
muffin.n.01 -> half__muffin.n.01
mushroom.n.05 -> half__mushroom.n.01
orange.n.01 -> half__orange.n.01
pastry.n.01 -> half

In [34]:
clam = Synset.get("clam.n.03")
for task in Task.all_objects():
  for transition in task.relevant_transitions:
    if clam in transition.input_synsets or clam in transition.output_synsets:
      print(task.name, transition.name, transition.input_synsets, transition.output_synsets)

make_seafood_stew-0 fish_stew <bddl.knowledge_base.orm.ManyToMany object at 0x7f11a8fc1c00> <bddl.knowledge_base.orm.ManyToMany object at 0x7f11a8fc1c90>
